# Disguised Intentions: How DBs Keep Offenses Guessing
Metric track | Author: [Miguel Duarte](https://github.com/miguelmendesduarte)

"_All warfare is based on deception. (...) When we are near, we must make the enemy believe we are far away; when far away, we must make him believe we are near._"

**- Sun Tzu, The Art of War**

---

# I. Introduction
In modern NFL defenses, **defensive backs** (DBs) are vital players in both coverage and pressure schemes. While their primary role is often in pass coverage, they also play a significant part in blitz packages. A well-executed DB blitz can disrupt the offense by confusing the quarterback and offensive line with unexpected pressure. Success in these situations relies on timing and the DB's ability to disguise their intentions. Disguising coverage is another essential skill for DBs. By showing one defensive look pre-snap and shifting to another post-snap, they can mislead quarterbacks and force poor decisions, leading to incompletions, sacks, or even turnovers.

To illustrate this dynamic, consider Jaire Alexander (#23), the Packers' cornerback, in a key play against the Vikings (Figure 1). With a clever disguise, Alexander blitzed and sacked Kirk Cousins for a safety, demonstrating how effective deception can impact the game.

<div style="display: flex; justify-content: center; align-items: center;">
    <img src="../assets/clips/jaire_alexander_sack.gif" width="640">
</div>

<figcaption style="text-align: center; font-size: 14px; color: gray;">Figure 1: Jaire Alexander sacks Kirk Cousins for a safety in Week 1 of the 2020 NFL season.</figcaption>

But how good was the disguise? Could the offense have predicted and adjusted their blocking to account for his blitz? The ability to disguise intentions can force the offense into a rushed decision or a missed block, which may not always be reflected in immediate statistics like sacks or turnovers, but can still significantly influence the outcome of a play. By quantifying these subtle influences, we can better understand how much of a DB's success comes from strategic deception, rather than pure physical skill.

This raises two key questions: **Can we quantitatively measure how effectively a DB disguises his intentions?** And more importantly, **how much does a successful disguise lead to tangible improvements in defensive outcomes?** In the following sections, we will explore these questions by analyzing data, applying a classification algorithm, creating a **disguise metric**, and examining the resulting insights.

Section II outlines the steps taken to clean and process the data, introduces the classification algorithm used, and explains the training process. In Section III, we present and analyze the results, addressing the key questions raised earlier. Finally, Section IV summarizes the main conclusions of this work and suggests potential directions for future research.

# II. Methodology
This study aims to quantitatively measure how effectively a DB disguises his intentions pre-snap and how this impacts defensive outcomes. To achieve this, a machine learning model is trained using pre-snap data to predict the likelihood of a blitz. The predicted blitz probabilities are then compared to the DB's post-snap actions (blitz or coverage) to assess the alignment between the predicted and actual outcomes.

## Data
The analysis focuses on cornerback behavior during pass plays, so all non-passing plays, QB kneels, and designed rollouts and runs are excluded. Wildcat formation plays are also removed, as they feature unconventional formations and player roles, introducing unnecessary variability. Plays with penalties are also excluded because penalties disrupt the natural flow of the play and can skew player actions.

Big Data Bowl 2025 focuses on pre-snap insights, so the analysis includes only pre-snap tracking data. All frames before the offense is set are excluded, as there is no need to analyze data from this period. To standardize the data, all plays are adjusted so that the offense faces right. The angles are rotated such that 0º always points to the right, with angles increasing counterclockwise. Speeds are also modified to reflect movement relative to the line of scrimmage (LOS): if a player is moving toward the LOS, their speed is assigned a negative value, while moving away from the LOS results in a positive speed. Lastly, the data is filtered to include only defensive backs (DBs) - specifically safeties and cornerbacks - eliminating all other positions from the analysis.

To train and evaluate the model, the data is split into two sets: weeks 1 through 5 (inclusive) for training and weeks 6 through 9 for testing. This results in **5351 plays in the training set and 3707 in the testing set**. For the training data, only the 1.5 seconds (15 frames) immediately before the ball is snapped are used, focusing on the players' movements just before the snap. Therefore, **all results come from the testing data**, ensuring the model is not used on the data it is trained on.

## Features
The following features were used to train the model. Features marked with an asterisk (*) were excluded from the training of the final model (details in the [appendix](#appendix)). Both **movement-based features** and **contextual features** were used. Movement features capture the players' immediate pre-snap actions (e.g., speed, direction, distance), which help identify whether a player is in position for a blitz. Additionally, **contextual features** consider situational factors (e.g., down, yards to go, score differential) - that help assess whether it makes sense to blitz in a given scenario.

- **yardsToGo**
- **down**
- **position**
- **x, y**
- **s** *
- **a**
- **dis**
- **o**
- **dir** *
- **distance_to_qb**: The distance between the player and the quarterback.
- **orientation_to_qb**: The player's orientation relative to the quarterback.
- **direction_to_qb**: The angle of player's motion to the quarterback. *
- **distance_to_line_of_scrimmage**: The distance between the player and the LOS.
- **yardsToEndzone**: The number of yards remaining to the end zone. *
- **scoreDifferential**: The difference in score between the two teams (negative is defense is behind).
- **timeRemainingInSeconds**: The amount of time remaining in the game.
- **defenders_near_LOS**: The number of defenders within 2 yards in depth and 7 yards in width of the LOS, with the LOS centered around the quarterback's position..
- **TEs_on_right, TEs_on_left**: The number of tight ends on the right and left sides of the quarterback.
- **FBs_on_right, FBs_on_left**: The number of fullbacks on the right and left sides of the quarterback.
- **RBs_on_right, RBs_on_left**: The number of running backs on the right and left sides of the quarterback.
- **distance_to_closest_opponent**: The distance to the nearest opposing player. *
- **position_of_closest_opponent**: The position of the closest opposing player.
- **orientation_to_closest_opponent**: The player's orientation relative to the closest opponent.

The target variable for the model is **wasInitialPassRusher**, which indicates whether the DB was a pass rusher (blitz) for the play (1 = blitz, 0 = no blitz).

## Model
Extreme Gradient Boosting (**XGBoost**) was chosen for this task due to its strong performance in supervised classification problems with large datasets. The model was trained using the following hyperparameters, selected after testing 144 combinations:

- **max_depth**: 9
- **n_estimators**: 300
- **min_child_weight**: 3
- **learning_rate**: 0.1

Key performance metrics:
- **Accuracy**: 97.14%
- **Precision**: 33.29%
- **Recall**: 22.18%
- **Log Loss**: 0.098

Although precision and recall may appear low at first glance, these results should not be viewed as poor. The main goal of the model is to predict probabilities, not just class labels. Furthermore, achieving high precision is inherently difficult because DBs are trained to mislead the opponent, making it harder for the model. Consequently, **quantifying uncertainty is essential**, allowing the model to express its confidence in its prediction. Therefore, **log loss** is the most appropriate metric, as it evaluates the accuracy of the model's probability estimates.

## Disguise Score
To evaluate how well a defensive back (DB) disguises his intentions, we calculate a **Disguise Score**. This metric is computed as follows:

1. **Calculate the Disguise Score for Each Frame**:      
   For each frame, we compute the disguise score as the absolute difference between the action taken by the DB (blitz = 1, coverage = 0) and the predicted probability of the DB blitzing.

    $$
    \text{Disguise Score (frame)} = \left| \text{Action} - \text{Predicted Blitz Probability} \right|
    $$

   - If the DB is predicted to blitz with a high probability, but the action is coverage (or vice versa), the disguise score will be higher, indicating a stronger disguise.

2. **Exponential Weighting**:    
   To prioritize frames closer to the snap, we apply an **exponential decay function** based on the time-to-snap for each frame. This ensures that **actions taken closer to the snap have a greater impact on the final disguise score**.

    $$
    \text{Weight (frame)} = e^{-k \cdot \text{T}}
    $$
   
   Where:
   - **_T_** is the time-to-snap, in seconds, and decreases as the play approaches the snap.
   - **_k_** is the decay constant, set to 0.4, controlling how quickly the weight decays as the time-to-snap increases.

2. **Final Disguise Score**:  
   The overall **Disguise Score** for a play is computed as the weighted average of the individual frame scores:

   $$
   \text{Disguise Score (play)} = \frac{\sum \text{(Weighted Disguise Scores for each frame)}}{\sum \text{(Weights)}}
   $$

### Interpretation of the Disguise Score
- **Higher Disguise Scores** indicate that the DB was successful at concealing his true intentions, making it harder for the offense to predict his actions.

- **Lower Disguise Scores** suggest that the DB’s actions were more predictable, allowing the offense to better anticipate his actions.

# III. Results

## Can We Quantitatively Measure How Well a DB Hides Their Intent?
Figure 2 illustrates the model in action during pre-snap of a 3rd down play between the Bears and Commanders (Week 6, 2022 NFL Season). Jaquan Brisker (#9) starts near the line of scrimmage, close to the ball, clearly indicating a blitz, which the model predicts. Meanwhile, Kyler Gordon (#6) aligns as though in man coverage on Washington's receiver (#2) but reveals his blitz intention just before the snap, which the model also detects. Both players blitz, with Brisker sacking Carson Wentz for an 8-yard loss. 

In terms of disguise, Brisker's actions are predictable, resulting in a **Disguise Score of 0.099**. In contrast, Gordon's intention is only slightly revealed right before the snap which makes him harder to read, leading to a high **Disguise Score of 0.971**, confirming the accuracy of both the model and the disguise score computation.

<div style="display: flex; justify-content: center; align-items: center; gap: 20px;">
    <img src="../assets/animations/2022101300_291_cut.gif" height="420">
    <img src="../assets/animations/2022101300_blitz_probs.gif" height="420">
</div>
<figcaption style="text-align: center; font-size: 14px; color: gray;">Figure 2: (<em>Left</em>) 3rd down play where C. Wentz is sacked at WAS 18 for -8 yards by J. Brisker (Commanders vs. Bears, Week 6, 2022 NFL Season). (<em>Right</em>) Pre-snap blitz probabilities for Jaquan Brisker and Kyler Gordon.</figcaption>

Figure 3 shows the scenario opposite from the previous play. Both Arizona DBs, Budda Baker (#3) and Byron Murphy (#7), align near the line of scrimmage, close to the ball, signaling a potential blitz pre-snap. Baker faces no offensive player nearby, while Murphy's position near Saints receiver #17 causes his blitz probability to fluctuate. The model predicts a high blitz probability for Baker throughout the pre-snap phase, while Murphy's probability varies. However, both players end up man-covering as the defense rushes only four players. Both DBs receive high Disguise Scores, with Baker at **0.977** and Murphy at **0.881**.

<div style="display: flex; justify-content: center; align-items: center; gap: 20px;">
    <img src="../assets/animations/2022102000_4205_cut.gif" height="420">
    <img src="../assets/animations/2022102000_4205_blitz_probs.gif" height="420">
</div>
<figcaption style="text-align: center; font-size: 14px; color: gray;">Figure 3: (<em>Left</em>) Tracking data showing Arizona DBs Budda Baker (#3) and Byron Murphy (#7) aligning near the defensive line (Cardinals vs. Saints, Week 7, 2022 NFL Season). (<em>Right</em>) Pre-snap blitz probabilities for Budda Baker and Byron Murphy.</figcaption>

## Disguise Score Rankings: Top and Bottom Performers
Table 1 highlights the NFL's most and least effective DBs at concealing their intentions during the 2022 season (Weeks 6-9). Jabrill Peppers (NE) leads with a score of 0.203, followed by Dane Belton, Jayron Kearse, and his teammate Kyle Dugger. Meanwhile, James Bradberry (PHI) and Patrick Peterson (MIN) score 0.000, indicating more predictable tendencies.

<figcaption style="text-align: center; font-size: 14px; color: gray;">Table 1: Top 10 and Bottom 10 DBs by Disguise Score (50+ Plays, 2022 NFL Season, Weeks 6-9).</figcaption>
<div style="display: flex; justify-content: center; align-items: center; gap: 20px;">
    <img src="../reports/figures/table_1_left.png" width="450">
    <img src="../reports/figures/table_1_right.png" width="450">
</div>

Throughout the season, Dugger, a standout safety for the Patriots, consistently demonstrated exceptional football instincts and high IQ. Coupled with his ability to disguise intentions and his athleticism, Dugger regularly made impactful plays. In contrast, lower-ranked players like Bradberry and Peterson, while elite, may have fewer opportunities to deceive opposing offenses. This may be due to their specific roles or the strategic approach of their teams. Additionally, team schemes focused on zone coverage or aggressive pass rushes can limit the opportunities for DBs to mask their intentions effectively. As a result, while these players excel in their particular roles, the nature of those roles may restrict their ability to achieve high disguise scores.

##  Who Are the Masters of Blitz Disguise?

<figcaption style="text-align: center; font-size: 14px; color: gray;">Table 2: DBs with +5 Blitzes Ranked by Mean Blitz Disguise Score (2022 NFL Season, Weeks 6-9).</figcaption>
<div style="display: flex; justify-content: center; align-items: center;">
    <img src="../reports/figures/table_2.png" width="800">
</div>

some text here...

<div style="display: flex; justify-content: center; align-items: center;">
    <img src="../assets/animations/2022110700_3059_cut.gif" width="420">
</div>
<figcaption style="text-align: center; font-size: 14px; color: gray;">Figure X: Blitz Disguise by Chris Harris (#19) during the Ravens vs. Saints with a <span style="font-weight: bold;">Disguise Score of 0.999981</span> (Week 9, 2022 NFL Season).</figcaption>

some text here...

## Blitz Disguise: Does It Increase Pressure on the QB?
Some text...

<div style="display: flex; justify-content: center; align-items: center;">
    <img src="../reports/figures/figure_5.png" width="640">
</div>
<figcaption style="text-align: center; font-size: 14px; color: gray;">Figure 5: Relationship Between DB Disguise Scores When Blitzing and QB Pressure and Hits (Percentage of Plays, 2022 NFL Season, Weeks 6-9).</figcaption>

## Limitations
While the model provides valuable insights into the effectiveness of DB blitz disguises, several limitations exist. First, the model uses a simplified representation of defensive behavior, which does not account for the full complexity of real-time decision-making on the field. DBs often use cues like QB cadence, audibles, and shifts to mask their intentions, but such factors are not captured in the data. Furthermore, the dataset used for this analysis is limited, covering only nine weeks of the 2022 NFL season. A broader dataset with more plays and teams would offer a more comprehensive view. Lastly, the model does not account for the intricacies of defensive schemes, such as zone or man coverage, impacting how well a DB can disguise their blitz.

# IV. Conclusion
This study presents a method for quantifying how effectively defensive backs disguise their intentions. The results reveal that some players excel at masking their blitzes, leading to higher disguise scores, while others, constrained by their roles or team schemes, struggle more. Ultimately, the Disguise Score offers valuable insights into the role of disguise in influencing plays and generating pressure on quarterbacks.

## Future Work
- **Temporal Analysis of Disguise Durations**: Future research should explore how early or late a DB's intentions are revealed, giving more weight to DBs who maintain their disguise longer. This analysis could also evaluate QBs’ hard counts, as those with better cadence may force earlier reveals.

- **Offensive Line Response**: Evaluating offensive lines' ability to recognize and react to disguised blitzes would provide insight into how well they handle defensive deception and which blocking schemes are most effective.

- **Opponent-Specific Strategies**: Analyzing how DBs adjust their disguises based on opponent tendencies could uncover how defensive strategies adapt to offensive behavior in similar game situations.

---
All code is available [here](https://github.com).

# Acknowledgements
Special thanks to Michael Lopez, Thompson Bliss, Ally Blake, Paul Mooney, Addison Howard, and the NFL staff for their contributions to the NFL Big Data Bowl 2025. I also want to thank the NFL and Kaggle for providing the data and making this experience possible.

As a passionate football fan in Portugal, I'm grateful for the opportunity to engage with the game through this competition.

# Appendix

## A.
...